In [1]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
from Arm_Lib import Arm_Device

Arm = Arm_Device()


In [3]:
import traitlets
import ipywidgets.widgets as widgets
import time
import threading
import inspect
import ctypes

origin_widget = widgets.Image(format='jpeg', width=320, height=240)
mask_widget = widgets.Image(format='jpeg',width=320, height=240)
result_widget = widgets.Image(format='jpeg',width=320, height=240)

image_container = widgets.HBox([origin_widget, mask_widget, result_widget])
# image_container = widgets.Image(format='jpeg', width=600, height=500)
display(image_container)

In [4]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res_async_raise(thread.ident, SystemExit)

In [5]:
def get_color(img,x,y,w,h):
    x=int(x)
    y=int(y)
    w=int(w)
    h=int(h)
    xx=int(x+w/2)
    yy=int(y+h/2)
    H = []
    color_name={}
    img = cv2.resize(img, (640, 480), )
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    for i in range(xx-10, xx+10):
        for j in range(yy-10, yy+10): H.append(HSV[j, i][0])
    H_min = min(H);H_max = max(H)
#     print(H_min,H_max)
    if H_min >= 0 and H_max <= 6 or H_min >= 179 and H_max <= 255: color_name['name'] = 'red'
    elif H_min >= 21 and H_max <= 51: color_name['name'] = 'yellow'
    elif H_min >= 52 and H_max <= 95: color_name['name'] = 'green'
    elif H_min >= 96 and H_max <= 124: color_name['name'] = 'blue'
    print(H_min,H_max)    
    return img, color_name

In [6]:
def my_get_color(img,x,y,w,h):
    my_color_name= 'none'
    img = cv2.resize(img, (640, 480), )
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    x=int(x+w/2)
    y=int(y+h/2)
    H=HSV[y, x][0]
    print(HSV[y, x])
#     print(H_min,H_max)
    if H>= 0 and H <= 6 or H >= 179 and H <= 255: my_color_name = 'red'
    elif H >= 21 and H <= 28: my_color_name = 'yellow'
    elif H >= 76 and H <= 87: my_color_name = 'green'
    elif H >= 101 and H <= 108: my_color_name= 'blue'
    print(my_color_name)
    return my_color_name

In [7]:
# Define variable parameters at different locations
# 定义不同位置的变量参数
look_at = [90, 130, 0, 0, 90, 30]
p_top = [90, 80, 50, 50, 270]

p_Yellow = [65, 22, 64, 56, 270]
p_Red = [118, 19, 66, 56, 270]

p_Green = [136, 66, 20, 29, 270]
p_Blue = [44, 66, 20, 28, 270]

p_gray = [90, 48, 35, 30, 270]



In [8]:
# Define control DOFBOT function, control No.1-No.6 servo，p=[S1,S2,S3,S4,S5,S6]
# 定义移动机械臂函数,同时控制1-6号舵机运动，p=[S1,S2,S3,S4,S5,S6]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# Define control DOFBOT function, control No.1-No.5 servo，p=[S1,S2,S3,S4,S5]
# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        elif id == 1 :
            Arm.Arm_serial_servo_write(id, p[i], int(3*s_time/4))
        else:
            Arm.Arm_serial_servo_write(id, p[i], int(s_time))
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# enable=1：clip，=0：release
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 145, 400)
    time.sleep(.5)

In [9]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [10]:
global g_state_arm
g_state_arm = 0

def ctrl_arm_move(index):
    arm_clamp_block(0)
    if index == 1:
        print("Yellow")

    elif index == 2:
        print("Red")

    elif index == 3:
        print("Green")

    elif index == 4:
        print("Blue")

    
    global g_state_arm
    g_state_arm = 0


In [11]:
# #Define the function corresponding to each number
# 数字功能定义
def number_action(index):
    if index == 1:
        # Grab yellow block
        # 抓取黄色的积木块

        arm_clamp_block(0)

    elif index == 2:
        # Grab red block
        # 抓取红色的积木块

        arm_clamp_block(0)

    elif index == 3:
        # Grab green block
        # 抓取绿色的积木块

        arm_clamp_block(0)

    elif index == 4:
        # Grab blue block
        # 抓取蓝色的积木块

        arm_clamp_block(0)

    
def put_down_block():

    arm_move_6(look_at, 1000)

    

In [12]:
import cv2
import numpy as np
def out_xy(img):
    x,y=0,0
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    kernel = np.ones((1, 5), np.uint8)
    t, binary = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY_INV)

    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, anchor=(2, 0), iterations=5)

    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    cv2.drawContours(img, contours, -1, (0, 0, 255), 5)
    if(len(contours)!=0):
        x,y,w,h = cv2.boundingRect(contours[0])
    return x,y

In [13]:
def start_move_arm(index):
    global g_state_arm
    if g_state_arm == 0:
        closeTid = threading.Thread(target = ctrl_arm_move, args = [index])
        closeTid.setDaemon(True)
        closeTid.start()
        
        g_state_arm = 1

In [14]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
# p_Yellow = [65, 22, 64, 56, 270]
# p_Red = [118, 19, 66, 56, 270]

# p_Green = [136, 66, 20, 29, 270]
# p_Blue = [44, 66, 20, 28, 270]

def calculate_loc(x,y,index):
    if(index==0):
        dest=[65, 22, 64, 56, 270]
    elif(index==1):
        dest=[118, 19, 66, 56, 270]
    elif(index==2):
        dest=[136, 66, 20, 29, 270]
    elif(index==3):
        dest=[44, 66, 20, 28, 270]
    data = pd.read_csv('123456.csv')
    X = data.iloc[:,0:2]
    Y = data.iloc[:,2:]
    neigh = KNeighborsClassifier(n_neighbors=3, weights='distance')
    neigh.fit(X, Y)
    a = neigh.predict([[x, y]])
    final=[]
    final = list(a[0])
    final.append(90)
    final.append(0)
    arm_move_6(final, 1000)
    arm_clamp_block(1)
    arm_move(p_top, 1000)
    arm_move(dest, 1000)
    arm_clamp_block(0)
    Arm.Arm_serial_servo_write(4,60, 500)
    arm_move(p_top, 1000)
    arm_move_6(look_at, 1000)
    time.sleep(2)
    return 0
    

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-wrtfauga because the default path (/home/jetson/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [15]:
import time
import cv2
def stable_xy(x,y):
    if(x+y!=0):
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        cap.set(4, 480)
        cap.set(5, 30)  
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
        time.sleep(1)
        a=[0,0]
        b=[0,0]
        while(1):
            ret, frame = cap.read()
            b[0:2]=out_xy(frame)
            res=abs(a[0]-b[0])+abs(a[1]-b[1])
            if(res<10):
                break                
            a[0:2]=b[0:2]
        x=a[0]
        y=a[1]
        return x,y


In [16]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets


cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(5, 30)  
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) 
# image.set(cv2.CAP_PROP_CONTRAST, 50)   
# image.set(cv2.CAP_PROP_EXPOSURE, 156)  

# Red is selected by default, and the program will automatically switch the color according to the color detected in the box
# 默认选择红色的,程序会自动根据方框中检测到的颜色切换颜色
# Red value
# 红色区间
red_color_lower = np.array([178, 70, 70])
red_color_upper = np.array([180, 255, 255])

# green value
# 绿色区间
green_color_lower = np.array([79, 24, 28])
green_color_upper = np.array([82, 253, 255])

# blue value
# #蓝色区间
blue_color_lower=np.array([103, 44, 44])
blue_color_upper = np.array([107, 250, 255])

# yellow value
# #黄色区间
yellow_color_lower = np.array([21, 52, 54])
yellow_color_upper = np.array([23, 255, 219])

# orange value
# #橙色区间
# color_lower = np.array([11, 43, 46])
# color_upper = np.array([25, 255, 255])


def Color_Recongnize():
    global zuobiao
    zuobiao=np.zeros([10,2])
    Arm.Arm_Buzzer_On(1)
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    count=0
    while(1):
        
        ret, frame = cap.read()
        if(ret==True):
        
            frame = cv2.resize(frame, (640, 480), )
        else:
            print(1)
            continue
        

        #cv2.imshow('Capture', frame)

        # change to hsv model
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        my_index=[]
        red_mask = cv2.inRange(hsv, red_color_lower, red_color_upper)
        blue_mask = cv2.inRange(hsv, blue_color_lower, blue_color_upper)
        green_mask = cv2.inRange(hsv, green_color_lower, green_color_upper)
        yellow_mask = cv2.inRange(hsv, yellow_color_lower, yellow_color_upper)
        my_index.append(np.sum(yellow_mask))
        my_index.append(np.sum(red_mask))
        my_index.append(np.sum(green_mask))
        my_index.append(np.sum(blue_mask))

        index=my_index.index(max(my_index))
        print(index)
        x,y=out_xy(frame)
        
        if(x+y!=0):
            if(count<=9):
                zuobiao[count,0]=x
                zuobiao[count,1]=y
                count+=1
            elif(count==10):
                zuobiao=zuobiao[np.lexsort(zuobiao.T)]
                x=zuobiao[4,0]
                y=zuobiao[4,1]
                print(x,y)
                calculate_loc(x,y,index)
                count=0
        

#         x,y=stable_xy(x,y)
#         calculate_loc(x,y)

        #cv2.imshow('Mask', mask)




        #cv2.imshow('Result', res)

        #     if cv2.waitKey(1) & 0xFF == ord('q'):
        #         break
        time.sleep(0.01)




    cap.release()
    #cv2.destroyAllWindows()

In [17]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

3
3
3
3
3
3
3
3
3
3
3
415.0 174.0
3
3
3
3
3
3
3
3
3
3
3
262.0 237.0
3
3
3
3
1
1
1
1
1
1
1
301.0 157.0
1
1
1
1
2
2
2
2
2
2
2
324.0 245.0
2
2
2
2
2
2
2
2
2
2
2
433.0 253.0
2
2
2
2
1
1
1
1
1
1
1
286.0 232.0
1
1
1
1
3
3
3
3
3
3
3
431.0 217.0


In [1]:
#End process, only need to execute this code at the end
# #结束进程，只有在结束时才需要执行此段代码
try:
    while True:
        time.sleep(.000001)
except KeyboardInterrupt:
    print(" Program closed! ")
    pass
stop_thread(thread1)

NameError: name 'time' is not defined

In [ ]:
cap.release()            